# Day 23

link: https://adventofcode.com/2022/day/23

ข้อนี้ถึงกระบวนการ simulate จะค่อนข้างซับซ้อนแต่ก็ทำตามได้ตรงๆ 
เราพบปัญหา performance นิดหน่อยกับ ruby เลยเป็นอีกข้อที่ต้องย้ายไปทำใน scala

ทีแรกเราอ่าน input แล้วเก็บเป็น Set ของตำแหน่ง (row, column)
แต่เพราะปัญหา performance เลยทำให้เราพยายามหา representation ที่ compact กว่า `[row, column]`

เราเลยแปลง (row, column) ไปเป็นเลขตัวเดียวด้วยฟังก์ชั่น (row, column) => row * C + column
ซึ่งจะใช้การได้ถ้าเรามั่นใจว่า ทุกค่าของ row และ column มีค่าอยู่ระหว่าง 0 ถึง C-1

เราเลือก C = 2^16 ซึ่งจะทำให้ค่าทั้งหมดถูก encode ลง 32-bit integer ได้ และใช้ 16 bit ทางซ้ายเก็บ row, 16 bit ทางขวาเก็บ column พอดี
ในทางปฏิบัติเราใช้บิตซ้ายสุดไม่ได้ ไม่งั้นจะต้องไปวุ่นวายกับจำนวนติดลบ ดังนั้นเราใช้ได้จริงๆ 15 bit
ตอนที่ initialize input เราก็บวก offset ไป 2^14 เพื่อให้จุด (0, 0) ไปอยู่ตรงกลางของ grid ขนาด 2^15 x 2^15 ที่เรามีอยู่

In [1]:
kernel.silent(true)

In [2]:
def elfEncode(x: Int, y: Int): Int = (x << 16) + y
def extractX(elf: Int): Int = elf >> 16
def extractY(elf: Int): Int = (elf << 16) >> 16

val C = 1 << 16

val input = scala.io.Source.fromFile("in23.txt").getLines.map(_.trim).toArray

val initialElves = (for {
  (line, i) <- input.zipWithIndex
    (c, j) <- line.zipWithIndex if c == '#'
  } yield elfEncode(i + C / 4, j + C / 4)
).toSet

## Part 1

เราสร้าง method `move` ซึ่งรับตำแหน่งของเอลฟ์ แล้วคำนวณหาตำแหน่งในรอบถัดไป

ขั้นแรก เราดูว่าเอลฟ์ในแต่ลำตำแหน่ง propose จะขยับไปตรงไหน
เราคำนวณว่าสามารถขยับไปได้ในทิศทางไหนบ้าง ถ้าไปได้ทุกทาง หรือไม่ได้เลยสักทาง เอลฟ์ก็จะอยู่ที่เดิม
ถ้าได้บางทิศทางก็ต้องเลือกทางแรก ตามลำดับที่โจทย์กำหนด เราเก็บ choice ของเอลฟ์แต่ละตัวไว้
ขณะเดียวกันก็เก็บว่าแต่ละตำแหน่งเป้าหมายถูกเลิอกกี่ครั้ง เราสามารถใช้ LongMap สำหรับข้อมูลทั้งสองก้อนนี้ได้

จากนั้นเราเช็คว่า choice ที่เอลฟ์เลือกนั้น unique หรือไม่ ถ้า unique ก็ขยับไปตามนั้น ถ้าไม่ก็อยู่ที่เดิม

พอได้ method นี้แหล้ว เราก็แค่เรียกมัน 10 ครั้ง แล้วคำนวณพื้นที่ ลบออกด้วยจำนวณเอลฟ์

In [3]:
import scala.collection.mutable

val directions = Array(
  (-C, -C - 1, -C + 1),
  ( C,  C - 1,  C + 1),
  (-1, -C - 1,  C - 1),
  ( 1, -C + 1,  C + 1)
)

def move(elves: Set[Int], step: Int) = {
  val proposes = new mutable.LongMap[Int]()
  val proposers = new mutable.LongMap[Int]().withDefaultValue(0)

  elves.foreach { current =>
    val canMoves = (0 until 4)
      .map(directions)
      .map {
        case (m1, m2, m3) =>
          !elves.contains(current + m1) &&
            !elves.contains(current + m2) &&
            !elves.contains(current + m3)
      }

    if (!canMoves.forall(identity) && !canMoves.forall(x => !x)) {
      val i = (0 until 4)
        .map(i => (i + step) % 4)
        .find(canMoves)
        .get
      val target = current + directions(i)._1
      proposes(current) = target
      proposers(target) += 1
    }

  }

  var moves = 0
  val nextElves = elves.map { current =>
    proposes.get(current) match {
      case Some(target) if proposers(target) == 1 =>
        moves += 1
        target
      case _ =>
        current
    }
  }

  (nextElves, moves)
}

def freeArea(elves: Set[Int]) = {
  val minR = elves.map(extractX).min
  val maxR = elves.map(extractX).max
  val minC = elves.map(extractY).min
  val maxC = elves.map(extractY).max

  (maxR - minR + 1) * (maxC - minC + 1) - elves.size
}

val res1 = Iterator
  .iterate((initialElves, 0, 1)) {
    case (elves, di, _) =>
      val (nextElves, moves) = move(elves, di)
      (nextElves, di + 1, moves)
  }
  .drop(10)
  .next()

println(freeArea(res1._1))

3877


ของแถม เราลอง render ดูด้วยว่า state ตอนจบหน้าตาเป็นยังไง

In [4]:
def render(elves: Set[Int]) = {
  val minR = elves.map(extractX).min
  val maxR = elves.map(extractX).max
  val minC = elves.map(extractY).min
  val maxC = elves.map(extractY).max
  (0 to maxR - minR).map { y =>
    (0 to maxC - minC).map { x =>
      if (elves.contains(elfEncode(y + minR, x + minC))) '#' else '.'
    }.mkString("")
  }.foreach(println)
}


render(res1._1)

................#..#.......#...#................................................
...........#.........#.......#.....#.#...#....#.#....#.................#........
......#........#........##.....###..........#....#.......##..##...##.#..........
........#...#...#.......#..#...........#......#.....#......#.............#......
.........#.#..#...#####...#.#.#.#.#..#...##..#..#.#....##.....#.#..#.#......#...
....#..#....#..#...##..#...#...#...#.#....#..#...#..#.......#..#.......##.#.....
..#......#....#.........#.#.#.....#.#..#.#...#.......#..#.##.#...#..#.#.........
.....#..#...#..###.#.#.#.#.#.#.#.#..#.#.#..#..#.###.#..#......#.#..#.#.###...#..
......#...###.#.#.#.#.###.#.#.#.#.#..#.#..##.......#..#.#.#.##.#..#...#.........
...##...#.#...#.#.####.#.#.#.#####.#..#.#....##.#.#.#.##.#.#..#.##.#.#.##.###...
...##.#..#.#.###...##.#.####.#....#..#.#.#.#...#.#.#.#..#....#..#.#.#.#..#...#..
....#......##..##.####.#.##.#.####.#..#.#.#.#.#.#...#..#..##...#...#.#..#.......
..#.....#.#.###.#..#..#.##.#

## Part 2

พาร์ทหลังเราแก้ให้ `move` นับจำนวนครั้งที่เอลฟ์ขยับเอาไว้ด้วย แล้วเรียกซ้ำไปเรื่อยๆ จนจำนวนนี้เป็น 0

ตอนแรกที่เขียนใน ruby เราพบว่า method นี้ใช้เวลาประมาณ 1 วินาทีต่อรอบ ซึ่งช้าเกินไปมาก แต่พอเปลี่ยนเป็น scala แล้ว ได้คำตอบทันที

In [5]:
val res2 = Iterator
  .iterate((initialElves, 0, 1)) {
    case (elves, di, _) =>
      val (nextElves, moves) = move(elves, di)
      (nextElves, di + 1, moves)
  }
  .dropWhile(_._3 > 0)
  .next()

println(res2._2)

982
